In [31]:
import utility_functions as utils
import importlib
import requests
from requests.exceptions import Timeout
import time
import json
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
importlib.reload(utils)

In [32]:
east_coast, west_coast = utils.load_east_west_json()

In [33]:
east_coast_filtered = utils.filter_data_between_years(east_coast, 1986, 1996)
west_coast_filtered = utils.filter_data_between_years(west_coast, 1986, 1996)

In [34]:
east_coast_artists = utils.get_set_of_artists(east_coast_filtered)
west_coast_artists = utils.get_set_of_artists(west_coast_filtered)

In [21]:
east_coast_artists

In [35]:
# Load the existing workbook
wb = load_workbook('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Artists.xlsx')
ws = wb['East Coast']

In [36]:
# Replace 'path_to_your_file.json' with the actual file path
with open('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Data/rolling_stone_100_east_coast.json', 'r') as file:
    data = json.load(file)

row = 2  # Start from the second row
for key, value in data.items():
    artist = value["artist"]
    song = value['title']
    year = value['date']
    ws[f'A{row}'] = artist
    ws[f'B{row}'] = song
    ws[f'C{row}'] = year
    row += 1

In [38]:
# Load the existing workbook
wb = load_workbook('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Artists.xlsx')
ws = wb['East Coast']

# Strings to look for and the fill color
strings_to_color = east_coast_artists
fill_color_green = 'AFBE8F'
fill_color_red = 'E64C54'

# Define the fill
fill_green = PatternFill(start_color=fill_color_green, end_color=fill_color_green, fill_type='solid')
fill_red = PatternFill(start_color=fill_color_red, end_color=fill_color_red, fill_type='solid')

for cell in ws['A']:
    if cell.value in strings_to_color:
        cell.fill = fill_green
    else:
        cell.fill = fill_red

# Save the workbook
wb.save('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Artists.xlsx')

In [30]:
# Load the existing workbook
wb = load_workbook('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Artists.xlsx')
ws = wb.active

# Strings to look for and the fill color
strings_to_color = west_coast_artists
fill_color_green = 'AFBE8F'
fill_color_red = 'E64C54'

# Define the fill
fill_green = PatternFill(start_color=fill_color_green, end_color=fill_color_green, fill_type='solid')
fill_red = PatternFill(start_color=fill_color_red, end_color=fill_color_red, fill_type='solid')

for cell in ws['A']:
    if cell.value in strings_to_color:
        cell.fill = fill_green
    else:
        cell.fill = fill_red

# Save the workbook
wb.save('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Artists.xlsx')

In [39]:
from openpyxl import load_workbook

# Load the existing workbook
wb = load_workbook('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Artists.xlsx')
ws = wb['East Coast']

row = 2
for i in east_coast_artists:
    ws[f'D{row}'] = i
    row += 1

wb.save('/Users/borosabel/Documents/Uni/Thesis/PopMIR/Artists.xlsx')

In [16]:
# Cleanup
for index, artist in enumerate(east_coast_artists):
    east_coast_artists[index] = utils.artist_cleanup(artist)

for index, artist in enumerate(west_coast_artists):
    west_coast_artists[index] = utils.artist_cleanup(artist)

In [5]:
east_coast_artists

In [235]:
genius = utils.get_genius_object()

In [14]:
east_coast_artists_json = utils.load_json('east_coast_artists.json')
west_coast_artists_json = utils.load_json('west_coast_artists.json')

In [6]:
# utils.save_json('west_coast_artists.json', west_coast_artists)
# utils.save_json('east_coast_artists.json', east_coast_artists)

In [230]:
for artist in west_coast_artists_json.keys():
    if west_coast_artists_json[artist] != 'SKIP':
        try:
            artist_object = genius.search_artist(artist, max_songs=1)
            artist_information = genius.artist(artist_object.id)['artist']

            if artist_information and west_coast_artists_json[artist] == '':
                temp = {
                    "id": artist_information['id']
                }
                new_key = artist_information['name']
                west_coast_artists_json[new_key] = temp
                del west_coast_artists_json[artist]
            else:
                print(f'--- SKIPPING: {artist} ---- \n')

        except AttributeError:
            print(f'--- ATTRITBUTE ERROR EXCEPTION AT: {artist} ---- \n')
            west_coast_artists_json[artist] = 'SKIP'
        except Timeout:
            print(f'--- TIMEOUT AT: {artist} ---- \n')
            west_coast_artists_json[artist] = 'SKIP'
        except requests.exceptions.RequestException as e:
            # Handle other requests exceptions
            print(f'--- REQUEST EXCEPTION AT: {artist}: {e} ---- \n')
            west_coast_artists_json[artist] = 'SKIP'
        finally:
            utils.save_json('west_coast_artists.json', west_coast_artists_json)

        time.sleep(1)

In [220]:
for artist in east_coast_artists_json.keys():
    if east_coast_artists_json[artist] != 'SKIP':
        try:
            artist_object = genius.search_artist(artist, max_songs=1)
            artist_information = genius.artist(artist_object.id)['artist']
    
            if artist_information and east_coast_artists_json[artist] == '':
                temp = {
                    "id": artist_information['id']
                }
                new_key = artist_information['name']
                east_coast_artists_json[new_key] = temp
                del east_coast_artists_json[artist]
            else:
                print(f'--- SKIPPING: {artist} ---- \n')
    
        except AttributeError:
            print(f'--- ATTRITBUTE ERROR EXCEPTION AT: {artist} ---- \n')
            east_coast_artists_json[artist] = 'SKIP'
        except Timeout:
            print(f'--- TIMEOUT AT: {artist} ---- \n')
            east_coast_artists_json[artist] = 'SKIP'
        except requests.exceptions.RequestException as e:
            # Handle other requests exceptions
            print(f'--- REQUEST EXCEPTION AT: {artist}: {e} ---- \n')
            east_coast_artists_json[artist] = 'SKIP'
        finally:
            utils.save_json('east_coast_artists.json', east_coast_artists_json)
            
        time.sleep(1)

In [245]:
a = genius.artist(west_coast_artists_json['N.W.A']['id'])['artist']

In [250]:
b = genius.artist_albums(west_coast_artists_json['N.W.A']['id'])['albums']

In [262]:
for artist in east_coast_artists_json.keys():
    if east_coast_artists_json[artist] != 'SKIP' and isinstance(east_coast_artists_json[artist], dict):
        try:
            artist_id = east_coast_artists_json[artist]['id']
            artist_albums = genius.search_albums(artist_id)

            if artist_albums and 'albums' not in east_coast_artists_json[artist]:
                for album in artist_albums:
                    if(1980 <= album['release_date_components']['year'] <= 1998):
                        temp = {
                            "id": album['id'],
                            "full_title": album['full_title'] 
                        }
                        east_coast_artists_json['artist']['album'] = temp
            else:
                print(f'--- SKIPPING: {artist} ---- \n')

        except AttributeError:
            print(f'--- ATTRITBUTE ERROR EXCEPTION AT: {artist} ---- \n')
            east_coast_artists_json[artist] = 'SKIP'
        except Timeout:
            print(f'--- TIMEOUT AT: {artist} ---- \n')
            east_coast_artists_json[artist] = 'SKIP'
        except requests.exceptions.RequestException as e:
            # Handle other requests exceptions
            print(f'--- REQUEST EXCEPTION AT: {artist}: {e} ---- \n')
            east_coast_artists_json[artist] = 'SKIP'
        finally:
            utils.save_json('east_coast_artists.json', east_coast_artists_json)

        time.sleep(1)